In [3]:
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [4]:
# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [17]:
print(dftrain.info)

<bound method DataFrame.info of         sex   age  n_siblings_spouses  parch      fare   class     deck  \
0      male  22.0                   1      0    7.2500   Third  unknown   
1    female  38.0                   1      0   71.2833   First        C   
2    female  26.0                   0      0    7.9250   Third  unknown   
3    female  35.0                   1      0   53.1000   First        C   
4      male  28.0                   0      0    8.4583   Third  unknown   
5      male   2.0                   3      1   21.0750   Third  unknown   
6    female  27.0                   0      2   11.1333   Third  unknown   
7    female  14.0                   1      0   30.0708  Second  unknown   
8    female   4.0                   1      1   16.7000   Third        G   
9      male  20.0                   0      0    8.0500   Third  unknown   
10     male  39.0                   1      5   31.2750   Third  unknown   
11   female  14.0                   0      0    7.8542   Third  unkn

In [21]:
sex = dftrain['sex'].unique()
print(sex)
display(dftrain.head())

['male' 'female']


,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [11]:
## Preprocess the data

In [9]:
import tensorflow as tf
#tf.random.set_seed(123)
fc = tf.feature_column
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']
def one_hot_cat_column(feature_name, vocab):
  return fc.indicator_column(
      fc.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  # Need to one-hot encode categorical features.
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(fc.numeric_column(feature_name,
                                           dtype=tf.float32))

In [12]:
# Use entire batch since this is such a small dataset.
NUM_EXAMPLES = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
    if shuffle:
      dataset = dataset.shuffle(NUM_EXAMPLES)
    # For training, cycle thru dataset as many times as need (n_epochs=None).
    dataset = (dataset
      .repeat(n_epochs)
      .batch(NUM_EXAMPLES))
    return dataset
  return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)

In [ ]:
## Train model

In [16]:
params = {
  'n_trees': 50,
  'max_depth': 3,
  'n_batches_per_layer': 1,
  # You must enable center_bias = True to get DFCs. This will force the model to
  # make an initial prediction before using any features (e.g. use the mean of
  # the training labels for regression or log odds for classification when
  # using cross entropy loss).
  'center_bias': True
}

est = tf.estimator.BoostedTrees(feature_columns, **params)
# Train model.
est.train(train_input_fn, max_steps=100)

# Evaluation.
results = est.evaluate(eval_input_fn)
clear_output()
pd.Series(results).to_frame()

AttributeError: module 'tensorflow.python.estimator.estimator_lib' has no attribute 'BoostedTrees'

In [ ]:
## Model interpretation and plotting

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns_colors = sns.color_palette('colorblind')